## Exercise 3

### Installation of ffmpeg on windows machine

https://www.ffmpeg.org/download.html#build-windows
following link to 
https://www.gyan.dev/ffmpeg/builds/

Four options available:

git full - built from master branch with a large set of libraries
git essentials - built from master branch with commonly-used libraries
release full - built from latest release branch with a large set of libraries
release essentials - built from latest release branch with commonly-used libraries downloaded the following package

Opted for the full one
https://www.gyan.dev/ffmpeg/builds/packages/ffmpeg-2021-08-18-git-dbe40478e2-full_build.7z

This also contains ffprobe

In [1]:
from os import walk

import pathlib

import sys

!{sys.executable} -m pip install ffprobe-python
!{sys.executable} -m pip install ffmpeg-python

from ffprobe import FFProbe
import ffmpeg

In [2]:
#Contains the results of the ffprobe analysis
class Report:
    def __init__(self, filePath):
        #Path of the file the report refers to
        self.filePath = filePath
        #Path of the fixed file after correction
        self.fixedFilePath = ''
        
        self.isOkAudioStream = True
        self.isOkVideoStream = True
        self.isOkContainer = True
        self.isOkVideoCodec = True
        self.isOkAudioCodec = True
        self.isOkFrameRate = True
        self.isOkAspectRatio = True
        self.isOkResolution = True
        self.isOkVideoBitRate = True
        self.isOkAudioBitRate = True
        self.report = ''
    #Checks the file and returns true if all checks pass    
    def isOk(self):
        return (self.isOkAudioStream and
            self.isOkVideoStream and
            self.isOkContainer and
            self.isOkVideoCodec and
            self.isOkAudioCodec and
            self.isOkFrameRate and
            self.isOkAspectRatio and
            self.isOkResolution and
            self.isOkVideoBitRate and
            self.isOkAudioBitRate)
        

#Checks whether the input file fullfills all requirements
def CheckFile(filePath):
    #Get the report
    report = Report(filePath)
    strReport = f'Report for file {filePath}\n'
    
    audioStream = None
    videoStream = None
    
    #extract the extension of the current file and ckeck it
    extension = pathlib.Path(report.filePath).suffix
    if extension != '.mp4':
        report.isOkContainer = False
        strReport += f'-- Wrong video container format. Expected mp4, found {extension}\n'
    
    #Run FFProbe on the file
    metadata = FFProbe(filePath)
    
    #extract audio and video streams from the ffprobe metadata
    for s in metadata.streams:
        if s.is_audio():
            audioStream = s
        elif s.is_video():
            videoStream = s
            
    #Check there is an audio stream and that it is as requirements
    if audioStream == None:
        report.isOkAudioStream = False
        strReport += '-- No audio stream in file\n'
    else:
        #Check audio codec
        if audioStream.codec() != 'aac':
            report.isOkAudioCodec = False
            strReport += f'-- Wrong codec. Expected aac, found {audioStream.codec()}\n'
        #Check audio bit rate  
        audioBitRate = int(audioStream.bit_rate) / 1000
        if (audioBitRate != 256):
            report.isOkAudioBitRate = False
            strReport += f'-- Wrong audio bit rate. Expected 256 kbps, found {audioBitRate} kbps\n'
        
    #Check there is an video stream and that it is as requirements
    if videoStream == None:
        report.isOkVideoStream = False
        strReport += '-- No video stream in file\n'
    else:
        #Check video codec
        if videoStream.codec() != 'h264':
            report.isOkVideoCodec = False
            strReport += f'-- Wrong codec. Expected h.264, found {videoStream.codec()}\n'
        #Check frame rate
        frameRate = round(videoStream.frames() / videoStream.duration_seconds())
        if frameRate != 25:
            report.isOkFrameRate = False
            strReport += f'-- Wrong frame rate. Expected 25, found {frameRate}\n'
        #Check aspect ratio
        aspectRatio = videoStream.frame_size()[0] / videoStream.frame_size()[1]
        if aspectRatio != 16/9:
            report.isOkAspectRatio = False
            strReport += f'-- Wrong aspect ratio. Expected 16/9 ({16/9}), found {aspectRatio}\n'
        #Check frame size
        if videoStream.frame_size() != (640, 360):
            report.isOkResolution = False
            strReport += f'-- Wrong resolution. Expected 640x360, found {videoStream.frame_size()[0]}x{videoStream.frame_size()[1]}\n'
        #Check video bit rate
        videoBitRate = int(videoStream.bit_rate) / 1000000
        if (videoBitRate < 10 or videoBitRate > 15):
            report.isOkVideoBitRate = False
            strReport += f'-- Wrong video bit rate. Expected between 10 and 15 Mbps, found {videoBitRate} Mbps\n'
    strReport += '---------------------------------------------\n\n'
    
    report.report = strReport
    return report





#Converts the file that does not fullfill the requirements using ffmpeg
def FixFile(report):
    if report.isOk():
        return
    
    #Set new file path
    extension = '.mp4'
    report.fixedFilePath = f'{report.filePath[0: -len(extension)]}_formatOK{extension}'
    
    #extract audio and video streams
    stream = ffmpeg.input(report.filePath)
    audio = stream.audio
    video = stream.video
    
    #set fps
    video = video.filter('fps', fps=25, round='up')     
    #set frame size and resolution
    video = video.filter('scale', width='640', height='360')
    
    #set video and audio codec, video and audio bitrate
    output = ffmpeg.output(audio, video, report.fixedFilePath, vcodec="libx264", acodec="aac", video_bitrate="12M", audio_bitrate="256K")    
    
    #save the file
    output.overwrite_output().run()
    

In [3]:
#input folder containing video files
filmFolderPath = "Exercise3_Films"

#all files in input folder
filenames = next(walk(filmFolderPath), (None, None, []))[2]

for f in filenames:
    #discard files created in previous runs of the algorithm
    if '_formatOK' in f:
        continue
        
    filePath = f'{filmFolderPath}\\{f}'
    
    #run check on file
    report = CheckFile(filePath)
    
    #print the results
    print(report.report)
    
    #if report says current file is OK, skip next step
    if report.isOk():
        continue
    
    #fix the file
    FixFile(report)
    
    #re-print the report on the fixed file
    report = CheckFile(report.fixedFilePath)

    #print report run on the newly created file
    print(report.report)
    

Report for file Exercise3_Films\Cosmos_War_of_the_Planets.mp4
-- Wrong audio bit rate. Expected 256 kbps, found 317.103 kbps
-- Wrong frame rate. Expected 25, found 30
-- Wrong aspect ratio. Expected 16/9 (1.7777777777777777), found 1.7740112994350283
-- Wrong resolution. Expected 640x360, found 628x354
-- Wrong video bit rate. Expected between 10 and 15 Mbps, found 2.989377 Mbps
---------------------------------------------


Report for file Exercise3_Films\Cosmos_War_of_the_Planets_formatOK.mp4
-- Wrong audio bit rate. Expected 256 kbps, found 245.851 kbps
---------------------------------------------


Report for file Exercise3_Films\Last_man_on_earth_1964.mov
-- Wrong video container format. Expected mp4, found .mov
-- Wrong codec. Expected aac, found pcm_s16le
-- Wrong audio bit rate. Expected 256 kbps, found 1536.0 kbps
-- Wrong codec. Expected h.264, found prores
-- Wrong frame rate. Expected 25, found 24
-- Wrong video bit rate. Expected between 10 and 15 Mbps, found 9.285191 M